In [ ]:
import numpy as np
import tensorflow as tf
import scipy as scp
import sklearn as skl

In [ ]:
batch_size = ... # the number of repetition * batch_size must be <= to the dataset size
nb_classes = ...

def data_loader_custom(X,augmentation_1 = lambda x : x,augmentation_2 = lambda x : x, nb_classes = nb_classes):
    
    def configure_for_performance(ds):
        ds = ds.shuffle(buffer_size=2*batch_size)
        ds = ds.batch(batch_size=batch_size)
        ds = ds.repeat()
        ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return ds
    
    
    X_aug1 = augmentation_1(X)
    
    X_aug2 = augmentation_2(X)   
    fake_labels = tf.data.Dataset.from_tensor_slices(np.zeros((len(X_aug1),nb_classes)))
    X = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(X_aug1), tf.data.Dataset.from_tensor_slices(X_aug2)))
    ds = tf.data.Dataset.zip(X,Y)   
    ds = configure_for_performance(ds)
    return ds

In [ ]:
avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)

class custom_fit(tf.keras.Model):
    
    def tf_unif_prior_crossentropy(pred1,pred2):
        N = batch_size
        C = nb_classes
        t_c = 0.1
        t_r = 0.1
        a = tf.nn.softmax(pred2/t_c, axis=0)
        b = tf.linalg.norm(a, ord=1,axis=1)
        c = a/tf.reshape(b,(N,-1))

        part11 = c
        d = tf.nn.softmax(pred1/t_r, axis=1)
        part21 = tf.reshape(tf.math.log(N/C *d/tf.linalg.norm(d, ord=1, axis=0)), (-1,C))
        loss_1 = -tf.math.reduce_sum( part11 * part21 , axis=-1)


        a = tf.nn.softmax(pred1/t_c)
        b = tf.linalg.norm(a, ord=1,axis=1)
        c = a/tf.reshape(b,(N,-1))
        part12 = c
        d = tf.nn.softmax(pred2/t_r, axis=1)
        part22 = tf.reshape(tf.math.log(N/C *d/tf.linalg.norm(d, ord=1, axis=0)),(-1,C))
        loss_2 = -tf.math.reduce_sum( part12*part22 , axis=-1)

        return tf.math.reduce_mean(1/2*(loss_1+loss_2))
    
    def train_step(self, data):
        input_data = data
        with tf.GradientTape() as tape:
            outputs = self(input_data, training=True) # forward pass 
            pred1 = outputs[0]
            pred2 = outputs[1]
            total_loss = self.tf_unif_prior_crossentropy(pred1,pred2)
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        avg_loss.update_state(total_loss)
        return {"loss": avg_loss.result()}
    
    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [avg_loss]